In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import os
import shap
import lime
from lime import lime_image
import cv2
from tensorflow.keras.models import Model


In [6]:

# Data directories (update these paths as needed)
train_dir = '/home/tanuj/Brain_MRI/brain_tumor_mri_dataset/Training'
val_dir = '/home/tanuj/Brain_MRI/brain_tumor_mri_dataset/Testing'

# Image Data Generator for Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [7]:

# Data Loading
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 8582 images belonging to 4 classes.
Found 1705 images belonging to 4 classes.


In [18]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


In [15]:

# CNN Model Architecture
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4, activation='softmax')  # 4 classes
    ])
    return model


In [ ]:

# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Training with tqdm Progress Bar
epochs = 30
history = model.fit(
    tqdm(train_generator),
    epochs=epochs,
    validation_data=val_generator
)

# Visualization - Training & Validation Accuracy
def plot_accuracy(history):
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.title('Training and Validation Accuracy')
    plt.show()

plot_accuracy(history)

# Grad-CAM Visualization (for Explainability)
def grad_cam(model, img_array, layer_name='conv2d_3'):
    grad_model = Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)[0]
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    return heatmap

# Display Grad-CAM
def display_gradcam(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150)) / 255.0
    img_array = np.expand_dims(img, axis=0)

    heatmap = grad_cam(model, img_array)

    plt.imshow(img)
    plt.imshow(heatmap, cmap='jet', alpha=0.5)
    plt.title('Grad-CAM Visualization')
    plt.axis('off')
    plt.show()

# LIME Integration
def lime_explainer(img_path, model):
    explainer = lime_image.LimeImageExplainer()
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150)) / 255.0

    explanation = explainer.explain_instance(img, model.predict, top_labels=4, hide_color=0, num_samples=1000)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)

    plt.imshow(temp)
    plt.imshow(mask, cmap='jet', alpha=0.5)
    plt.title('LIME Visualization')
    plt.axis('off')
    plt.show()

# SHAP Integration
def shap_explainer(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150)) / 255.0
    img_array = np.expand_dims(img, axis=0)

    explainer = shap.GradientExplainer(model, img_array)
    shap_values = explainer.shap_values(img_array)

    shap.image_plot(shap_values, img_array)

# Example Usage:
# display_gradcam('path_to_example_image.jpg', model)
# lime_explainer('path_to_example_image.jpg', model)
# shap_explainer('path_to_example_image.jpg', model)
